In [2]:
# Import Selenium libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Import BeautifulSoup
from bs4 import BeautifulSoup

#Import Pandas and other libraries
import pandas as pd
import time
import random

def SelectUserAgent():
  user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; CrOS x86_64 14541.0.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) nBrowser/1.0.0 Chrome/121.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/121.0.6167.18 Safari/537.36'
    ]

  for _ in user_agent_list:
    user_agent = random.choice(user_agent_list)
    return user_agent

def IniciarChrome():
    options = Options()
    options.add_argument(f"user-agent={SelectUserAgent()}")
    options.add_argument("--start-maximized")
    options.add_argument("--headless")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-web-security")#
    options.add_argument("--disable-notifications")
    options.add_argument("--ignore-certificate-errors-spki-list")
    options.add_argument("--no-sandbox")
    options.add_argument("--allow-running-insecure-content")
    options.add_argument("--no-default-browser-check")
    options.add_argument("--no-first-run")
    options.add_argument("--no-proxy-server")
    options.add_argument("--disable-blink-features=AutomationControlled")

    # Parámetros a omitir en el inicio de ChromeDriver
    exp_opt = [
    'enable-automation',
    'ignore-certificate-errors',
    'enable-logging'
    ]
    options.add_experimental_option("excludeSwitches", exp_opt)

    # Parámetros que definen preferencias en ChromeDriver
    prefs = {
    "profile.default_content_setting_value.notifications": 2,
    "intl.accept_languages": ["es-ES", "es"],
    "credentials_enable_service": False
    }                                
    
    options.add_experimental_option("prefs", prefs)

    s = Service(r'C:\Users\Administrator\Desktop\chromedriver-win64\chromedriver.exe')

    driver = webdriver.Chrome(service = s, options = options)
    
    return driver

start_time = time.time()

Data = {'Name':[],
        'LinkedIn':[],
        'Title':[],
        'Company':[],
        'Location':[],
        'Email':[]}

url = 'https://app.apollo.io/#/login'
email = 'lucas.lopez1@udea.edu.co'
password = 'tCdfxQ5mq3@V2*Z'

button_classes = ['zp-button', 'zp_zUY3r', 'zp_hLUWg', 'zp_n9QPr', 'zp_B5hnZ', 'zp_MCSwB', 'zp_IYteB']

driver = IniciarChrome()
driver.get(url)

login_email = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, "zp_bWS5y")))
login_email.send_keys(email)

login_password = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "current-password")))
login_password.send_keys(password)
login_password.send_keys(Keys.ENTER)

WebDriverWait(driver, 20).until(EC.url_changes(url))

for num_pag in range(1,7):

    time.sleep(1)
    
    url = f'https://app.apollo.io/#/people?finderViewId=5b8050d050a3893c382e9360&contactLabelIds[]=65b302da51ee0a0001e28a85&prospectedByCurrentTeam[]=yes&page={num_pag}'

    driver.get(url)
    
    table = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.TAG_NAME, "table")))
    
    tbodies = table.find_elements(By.TAG_NAME,'tbody')
    
    for tbody in tbodies:
        try:
            ActionChains(driver)\
            .scroll_to_element(tbody)\
            .perform()
            Data['Name'].append(tbody.find_element(By.TAG_NAME,'a').text)
            try: Data['LinkedIn'].append(tbody.find_element(By.CLASS_NAME,'zp_I1ps2').find_element(By.TAG_NAME,'a').get_attribute('href'))
            except: Data['LinkedIn'].append('N/A')
            Data['Title'].append(tbody.find_element(By.CLASS_NAME,'zp_Y6y8d').text)
            Data['Company'].append(tbody.find_element(By.CLASS_NAME,'zp_WM8e5').text)
            Data['Location'].append(tbody.find_elements(By.CLASS_NAME,'zp_Y6y8d')[1].text)
            try: 
                button = tbody.find_element(By.CSS_SELECTOR, '.' + '.'.join(button_classes))
                button.click()
                Data['Email'].append(driver.find_element(By.CLASS_NAME, 'zp_t08Bv').text)
                button.click()
            except:
                Data['Email'].append('N/A')
                pass
            WebDriverWait(driver, 20).until_not(EC.visibility_of_element_located((By.CLASS_NAME, 'zp_DMnwE')))
        except StaleElementReferenceException:
            continue

driver.quit()

df = pd.DataFrame(Data)

end_time = time.time()

print(f'Tiempo Total:{end_time-start_time}')

df

Tiempo Total:147.62835025787354


,Name,LinkedIn,Title,Company,Location,Email
0,Joshua Gould,http://www.linkedin.com/in/joshuagould100,Football Coach,SHORE,Australia,jgould@shore.nsw.edu.au
1,Mark Sleiman,http://www.linkedin.com/in/mark-sleiman,Football Coach,Charlton Athletic Football Club,"London, United Kingdom",mark.sleiman@cafc.co.uk
2,Jon Underwood,http://www.linkedin.com/in/jon-underwood-44824680,Football Coach,Eton College,United Kingdom,jon.underwood@etoncollege.com
3,Paul Ifill,http://www.linkedin.com/in/paul-ifill-079a8584,Football Coach,Christchurch United FC,"Wellington, New Zealand",paul@cufc.co.nz
4,Shaun Bragg,http://www.linkedin.com/in/shaun-bragg-a08765221,Football Coach,Kettering Town Football Club,"Northampton, United Kingdom",shaun.bragg@ketteringtownfc.com
...,...,...,...,...,...,...
118,Avinash Urs,http://www.linkedin.com/in/avinash-urs-66b093146,Football Coach,JSW Bengaluru Football Club Private Limited,"Bengaluru, India",avinash.urs@bengalurufc.com
119,Kevin Johnson,http://www.linkedin.com/in/kevin-johnson-13b7b038,Football Coach,Pace Academy,"Atlanta, Georgia",kevin.johnson@paceacademy.org
120,Sos Lukio,http://www.linkedin.com/in/sos-lukio-319934184,Football Coach,Chelsea Football Club,"England, United Kingdom",sos.lukio@chelseafc.com
121,Jacqui Driscoll,http://www.linkedin.com/in/jacqui-driscoll-7b9...,Football Coach,THE LEEDS UNITED FOUNDATION,"Leeds, United Kingdom",jacqui.driscoll@coopacademies.co.uk
